<a href="https://colab.research.google.com/github/miczkejedrzej/MNLP-project-1/blob/main/projetNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import of the training data

from google.colab import files
uploaded = files.upload()

Saving [MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv to [MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv


In [3]:
!pip install wikidata --quiet

import pandas as pd
from wikidata.client import Client
import requests

In [4]:
df_train = pd.read_csv('/content/[MNLP 2025 HW1] train set [PUBLIC] - train_cleaned.tsv', sep = '\t')

In [70]:
WIKIDATA_SPARQL_URL = "https://query.wikidata.org/sparql"

def get_superclass(qid):
    """ get the superclass to which qid is a subclass of """
    query = f"""
    SELECT ?superclass WHERE {{
      wd:{qid} wdt:P279 ?superclass.
    }}
    LIMIT 1 """
    #the first superclass is taken if there are several superclasses
    headers = {
        "Accept": "application/sparql-results+json",
        "User-Agent": "ExplorateurWikidata/0.1 (truc@truc.com)"
    }
    response = requests.get(WIKIDATA_SPARQL_URL, params={"query": query}, headers=headers)
    data = response.json()
    bindings = data["results"]["bindings"]
    if not bindings:
        return None
    superclass_uri = bindings[0]["superclass"]["value"]
    return superclass_uri.split("/")[-1]

def get_class_path(qid):
    """ find recursively the path to a "root" class"""
    path = [qid]
    current = qid
    while True:
        parent = get_superclass(current)
        if parent is None or parent in path:
            break
        path.append(parent)
        current = parent
    return path

def extract_entity_id(url):
    return url.strip().split("/")[-1]

def get_depth(url):
    qid = extract_entity_id(url)
    path_qids = get_class_path(qid)
    return len(path_qids)


In [79]:
df_subset = df_train.sample(n=20, random_state = 42)
df_subset["subclass_depth"] = df_subset["item"].apply(get_depth)

In [80]:
df_subset

,item,name,description,type,category,subcategory,label,subclass_depth
1703,http://www.wikidata.org/entity/Q17020543,LGBTQ Sports Hall of Fame,hall of fame established in 2013 to honor LGBT...,entity,sports,sport,cultural agnostic,1
5448,http://www.wikidata.org/entity/Q6735853,Maimal caste,Inland fishermen group of Bengal. Indigenous t...,entity,gestures and habits,tradition,cultural exclusive,1
5058,http://www.wikidata.org/entity/Q3513409,Congolese Round Table Conference,1960 meeting between Belgian and Congolese lea...,entity,history,historical event,cultural exclusive,1
1149,http://www.wikidata.org/entity/Q6545380,lifeguard tower,used at recreational beaches or public swimmin...,concept,architecture,building,cultural agnostic,7
432,http://www.wikidata.org/entity/Q4162339,Gay male blood donor controversy,policy on blood donation,concept,politics,policy,cultural agnostic,6
2292,http://www.wikidata.org/entity/Q2748526,Cecil O. Samuelson,American University president,entity,philosophy and religion,religious leader,cultural exclusive,1
964,http://www.wikidata.org/entity/Q217175,architectural order,"styles of classical architecture, most readily...",concept,architecture,architectural style,cultural representative,8
5518,http://www.wikidata.org/entity/Q1614911,Thai greeting,popular Thai greeting,concept,gestures and habits,greeting,cultural exclusive,10
4482,http://www.wikidata.org/entity/Q10259,Holi,Hindu spring festival of colors,entity,music,music festival,cultural representative,1
6032,http://www.wikidata.org/entity/Q16513,Alberche,river in Spain,entity,geography,river,cultural exclusive,1
